缺失值填补

In [1]:
import os
import pandas as pd
import torch
from sklearn.base import BaseEstimator, TransformerMixin
from autogluon.multimodal import MultiModalPredictor

class AutoMMIterativeImputer(BaseEstimator, TransformerMixin):
    def __init__(self, max_iter=100, random_state=0):
        self.max_iter = max_iter
        self.random_state = random_state
        self.num_gpus = torch.cuda.device_count()

    def fit(self, X, y=None):
        self.feature_names = X.columns[:-2]  # 排除最后两列
        return self

    def transform(self, X):
        # 设置环境变量以使用所有可用的GPU
        os.environ['CUDA_VISIBLE_DEVICES'] = ','.join(map(str, range(self.num_gpus)))

        output_csv = 'imputation_iterations.csv'
        
        # 初始化CSV文件
        iteration_data = pd.DataFrame(columns=['iteration', 'filled_column', 'missing_values_before', 'missing_values_after'])
        iteration_data.to_csv(output_csv, index=False, encoding='utf-8-sig')
        
        for i in range(self.max_iter):
            print(f"Iteration {i + 1}")
            filled_any = False
            for col in self.feature_names:
                missing_values_before = X[col].isna().sum()
                
                if missing_values_before == 0:
                    continue
                
                print(f"Filling missing values in column: {col}")

                train_data = X.dropna(subset=[col])
                test_data = X[X[col].isna()]

                if len(test_data) == 0:
                    continue

                predictor = MultiModalPredictor(label=col, problem_type='regression')
                train_features = train_data.drop(columns=[col])
                test_features = test_data.drop(columns=[col])
                
                # 训练模型
                predictor.fit(train_data)
                
                predictions = predictor.predict(test_features)
                
                # 处理负值，将负值修改为0
                predictions = predictions.apply(lambda x: max(x, 0))
                
                X.loc[test_data.index, col] = predictions
                
                missing_values_after = X[col].isna().sum()

                # 记录当前列的填充效果
                iteration_info = {
                    'iteration': i + 1,
                    'filled_column': col,
                    'missing_values_before': missing_values_before,
                    'missing_values_after': missing_values_after
                }
                iteration_df = pd.DataFrame([iteration_info])
                iteration_df.to_csv(output_csv, mode='a', header=False, index=False, encoding='utf-8-sig')

                if missing_values_after < missing_values_before:
                    filled_any = True

            if not filled_any:
                print("No more missing values to fill. Stopping iterations.")
                break

        return X

# 重新加载数据
file_path = '数据集 最终使用.csv'
df = pd.read_csv(file_path, header=None, keep_default_na=False, na_values=[""])

# 使用第四行作为列名
df.columns = df.iloc[3]
column_names = df.iloc[3]

# 删除前三行和作为列名的第四行
df = df.drop([0, 1, 2, 3])

# 使用自定义的迭代填充器填充缺失值
imputer = AutoMMIterativeImputer(max_iter=100)
df_filled = imputer.fit_transform(df)

# 添加列名回到数据集中
df_filled.columns = column_names

# 保存填充后的数据
output_path = '数据集 补充完整.csv'
df_filled.to_csv(output_path, index=False, encoding='utf-8-sig')

print(f"Filled data saved to {output_path}")

No path specified. Models will be saved in: "AutogluonModels\ag-20240731_070510"
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.9.19
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22621
CPU Count:          16
Pytorch Version:    2.3.1+cu121
CUDA Version:       12.1
Memory Avail:       4.11 GB / 15.73 GB (26.2%)
Disk Space Avail:   209.72 GB / 260.51 GB (80.5%)


Iteration 1
Filling missing values in column: Cellulose content



AutoMM starts to create your model. ✨✨✨

To track the learning progress, you can open a terminal and launch Tensorboard:
    ```shell
    # Assume you have installed tensorboard
    tensorboard --logdir f:\大模型项目\AutogluonModels\ag-20240731_070510
    ```

Seed set to 0
e:\Anaconda3\envs\myenv\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
GPU Count: 1
GPU Count to be Used: 1
GPU 0 Name: NVIDIA GeForce RTX 3050 Laptop GPU
GPU 0 Memory: 0.13GB/4.0GB (Used/Total)

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3050 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_p

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

...

Predicting: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

AutoMM has created your model. 🎉🎉🎉

To load the model, use the code below:
    ```python
    from autogluon.multimodal import MultiModalPredictor
    predictor = MultiModalPredictor.load("f:\大模型项目\AutogluonModels\ag-20240731_224041")
    ```

If you are not satisfied with the model, try to increase the training time, 
adjust the hyperparameters (https://auto.gluon.ai/stable/tutorials/multimodal/advanced_topics/customization.html),
or post issues on GitHub (https://github.com/autogluon/autogluon/issues).




Predicting: |          | 0/? [00:00<?, ?it/s]

Iteration 2
No more missing values to fill. Stopping iterations.
Filled data saved to 数据集 补充完整.csv


评估填补效果

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score, mean_squared_error
from autogluon.multimodal import MultiModalPredictor
import os

# 设置随机种子以确保结果可重复
np.random.seed(42)

# 创建一个目录来保存预测结果
output_dir = 'prediction_results'
os.makedirs(output_dir, exist_ok=True)

# 加载补全后的数据集
file_path_filled = r'F:\大模型项目\Paper_1\第一部分_数据集填补\数据集 补充完整.csv'
df_filled = pd.read_csv(file_path_filled)

# 加载原始数据集
file_path_original = r'F:\大模型项目\Paper_1\第一部分_数据集填补\数据集 最终使用.csv'
df_original = pd.read_csv(file_path_original, header=None, keep_default_na=False, na_values=[""])

# 使用第四行作为列名
df_original.columns = df_original.iloc[3]
df_original = df_original.drop([0, 1, 2, 3])

# 确保原始数据类型匹配
for col in df_original.columns:
    if col in df_filled.columns:
        df_filled[col] = df_filled[col].astype(df_original[col].dtypes)

# 初始化评估结果
evaluation_results = []

# 从第七列到倒数第三列
for target_col in df_filled.columns[6:-2]:
    print(f"Processing column: {target_col}")
    
    # 找到原始数据中的非缺失值索引并与补全数据集的索引对齐
    non_missing_indices_original = df_original[target_col].dropna().index
    non_missing_indices_filled = df_filled.index.intersection(non_missing_indices_original)
    
    # 随机选择30%的原始数据进行删除
    num_to_delete = int(len(non_missing_indices_filled) * 0.3)
    indices_to_delete = np.random.choice(non_missing_indices_filled, num_to_delete, replace=False)
    
    # 创建训练和测试集
    X_train = df_filled.copy()
    y_true = X_train.loc[indices_to_delete, target_col]
    
    # 将目标列的缺失值设置为NaN
    X_train.loc[indices_to_delete, target_col] = np.nan
    
    # 使用MultiModalPredictor填补缺失值
    predictor = MultiModalPredictor(label=target_col, problem_type='regression')
    train_data = X_train.dropna(subset=[target_col])
    test_data = X_train.loc[indices_to_delete]
    
    # 训练模型
    predictor.fit(train_data)
    
    # 预测缺失值
    y_pred = predictor.predict(test_data)
    
    # 评估预测效果
    r2 = r2_score(y_true, y_pred)
    rmse = mean_squared_error(y_true, y_pred, squared=False)
    
    # 打印当前列的评估结果
    print(f"Column: {target_col}, R²: {r2:.4f}, RMSE: {rmse:.4f}")
    
    evaluation_results.append({
        'column': target_col,
        'r2': r2,
        'rmse': rmse
    })
    
    # 创建并保存预测结果的 DataFrame
    prediction_df = pd.DataFrame({
        'index': indices_to_delete,
        'y_true': y_true.values,
        'y_pred': y_pred
    })
    
    # 定义保存路径
    prediction_csv_path = os.path.join(output_dir, f'{target_col}_predictions.csv')
    
    # 保存到 CSV 文件
    prediction_df.to_csv(prediction_csv_path, index=False, encoding='utf-8-sig')
    
    print(f"Prediction results for column '{target_col}' saved to {prediction_csv_path}")

# 将评估结果转换为DataFrame
evaluation_df = pd.DataFrame(evaluation_results)

# 将评估结果保存到txt文件
output_txt_path = 'evaluation_results.txt'
evaluation_df.to_csv(output_txt_path, sep='\t', index=False, encoding='utf-8-sig')

print(f"Evaluation results saved to {output_txt_path}")
print(f"All prediction results saved in the directory '{output_dir}'")

No path specified. Models will be saved in: "AutogluonModels\ag-20241222_032248"


Processing column: Cellulose content


=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.9.21
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.26100
CPU Count:          16
Pytorch Version:    2.4.1+cu121
CUDA Version:       12.1
Memory Avail:       6.64 GB / 15.73 GB (42.2%)
Disk Space Avail:   161.81 GB / 260.51 GB (62.1%)

AutoMM starts to create your model. ✨✨✨

To track the learning progress, you can open a terminal and launch Tensorboard:
    ```shell
    # Assume you have installed tensorboard
    tensorboard --logdir f:\大模型项目\AutogluonModels\ag-20241222_032248
    ```

Seed set to 0
GPU Count: 1
GPU Count to be Used: 1
GPU 0 Name: NVIDIA GeForce RTX 3050 Laptop GPU
GPU 0 Memory: 0.13GB/4.0GB (Used/Total)

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3050 Laptop GPU') that has Ten

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

...

Predicting: |          | 0/? [00:00<?, ?it/s]

f:\大模型项目\.conda\lib\site-packages\autogluon\multimodal\utils\checkpoint.py:63: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  avg_state_dict = torch.load(checkpoint_paths[0],

Predicting: |          | 0/? [00:00<?, ?it/s]

Column: grain size, R²: 0.9511, RMSE: 0.1233
Prediction results for column 'grain size' saved to prediction_results\grain size_predictions.csv
Evaluation results saved to evaluation_results.txt
All prediction results saved in the directory 'prediction_results'


f:\大模型项目\.conda\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
